*아래 링크를 통해 이 노트북을 주피터 노트북 뷰어(nbviewer.jupyter.org)로 보거나 구글 코랩(colab.research.google.com)에서 실행할 수 있습니다.*

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://nbviewer.jupyter.org/github/rickiepark/nlp-with-pytorch/blob/master/chapter_4/Chapter-4-In-Text-Examples.ipynb"><img src="https://jupyter.org/assets/main-logo.svg" width="28" />주피터 노트북 뷰어로 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/nlp-with-pytorch/blob/master/chapter_4/Chapter-4-In-Text-Examples.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
</table>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

seed = 1337

torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

### 코드 4-1

In [2]:
class MultilayerPerceptron(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        """
        매개변수:
            input_dim (int): 입력 벡터 크기
            hidden_dim (int): 첫 번째 Linear 층의 출력 크기
            output_dim (int): 두 번째 Linear 층의 출력 크기
        """
        super(MultilayerPerceptron, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x_in, apply_softmax=False):
        """MLP의 정방향 계산
        
        매개변수:
            x_in (torch.Tensor): 입력 데이터 텐서
                x_in.shape는 (batch, input_dim)입니다.
            apply_softmax (bool): 소프트맥스 활성화 함수를 위한 플래그
                크로스-엔트로피 손실을 사용하려면 False로 지정해야 합니다.
        반환값:
            결과 텐서. tensor.shape은 (batch, output_dim)입니다.
        """
        intermediate = F.relu(self.fc1(x_in))
        output = self.fc2(intermediate)
        
        if apply_softmax:
            output = F.softmax(output, dim=1)
        return output

### 코드 4-2

In [3]:
batch_size = 2 # 한 번에 입력할 샘플 개수
input_dim = 3
hidden_dim = 100
output_dim = 4

# 모델 생성
mlp = MultilayerPerceptron(input_dim, hidden_dim, output_dim)
print(mlp)

MultilayerPerceptron(
  (fc1): Linear(in_features=3, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=4, bias=True)
)


### 코드 4-3

In [4]:
def describe(x):
    print("타입: {}".format(x.type()))
    print("크기: {}".format(x.shape))
    print("값: \n{}".format(x))

In [5]:
# 입력
x_input = torch.rand(batch_size, input_dim)
describe(x_input)

타입: torch.FloatTensor
크기: torch.Size([2, 3])
값: 
tensor([[0.8329, 0.4277, 0.4363],
        [0.9686, 0.6316, 0.8494]])


In [6]:
y_output = mlp(x_input, apply_softmax=False)
describe(y_output)

타입: torch.FloatTensor
크기: torch.Size([2, 4])
값: 
tensor([[-0.2456,  0.0723,  0.1589, -0.3294],
        [-0.3497,  0.0828,  0.3391, -0.4271]], grad_fn=<AddmmBackward>)


### 코드 4-4

In [7]:
y_output = mlp(x_input, apply_softmax=True)
describe(y_output)

타입: torch.FloatTensor
크기: torch.Size([2, 4])
값: 
tensor([[0.2087, 0.2868, 0.3127, 0.1919],
        [0.1832, 0.2824, 0.3649, 0.1696]], grad_fn=<SoftmaxBackward>)


### 코드 4-13

In [9]:
class MultilayerPerceptron(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        """
        매개변수:
            input_dim (int): 입력 벡터 크기
            hidden_dim (int): 첫 번째 Linear 층의 출력 크기
            output_dim (int): 두 번째 Linear 층의 출력 크기
        """
        super(MultilayerPerceptron, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x_in, apply_softmax=False):
        """MLP의 정방향 계산
        
        매개변수:
            x_in (torch.Tensor): 입력 데이터 텐서
                x_in.shape는 (batch, input_dim)입니다.
            apply_softmax (bool): 소프트맥스 활성화 함수를 위한 플래그
                크로스-엔트로피 손실을 사용하려면 False로 지정해야 합니다.
        반환값:
            결과 텐서. tensor.shape은 (batch, output_dim)입니다.
        """
        intermediate = F.relu(self.fc1(x_in))
        output = self.fc2(F.dropout(intermediate, p=0.5))
        
        if apply_softmax:
            output = F.softmax(output, dim=1)
        return output

batch_size = 2 # 동시에 입력될 샘플의 개수
input_dim = 3
hidden_dim = 100
output_dim = 4

# 모델 생성
mlp = MultilayerPerceptron(input_dim, hidden_dim, output_dim)
print(mlp)

y_output = mlp(x_input, apply_softmax=False)
describe(y_output)

MultilayerPerceptron(
  (fc1): Linear(in_features=3, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=4, bias=True)
)
Type: torch.FloatTensor
Shape/size: torch.Size([2, 4])
Values: 
tensor([[ 0.0193,  0.0275,  0.2319,  0.3032],
        [-0.5323,  0.3183,  0.4194, -0.0205]], grad_fn=<AddmmBackward>)


### 코드 4-14

In [10]:
batch_size = 2
one_hot_size = 10
sequence_width = 7
data = torch.randn(batch_size, one_hot_size, sequence_width)
conv1 = nn.Conv1d(in_channels=one_hot_size, out_channels=16, kernel_size=3)
intermediate1 = conv1(data)
print(data.size())
print(intermediate1.size())

torch.Size([2, 10, 7])
torch.Size([2, 16, 5])


### 코드 4-15

In [11]:
conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3)
conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)

intermediate2 = conv2(intermediate1)
intermediate3 = conv3(intermediate2)

print(intermediate2.size())
print(intermediate3.size())

torch.Size([2, 32, 3])
torch.Size([2, 64, 1])


In [12]:
y_output = intermediate3.squeeze()
print(y_output.size())

torch.Size([2, 64])


In [13]:
intermediate2.mean(dim=0).mean(dim=1).sum()

tensor(-0.0493, grad_fn=<SumBackward0>)

### 코드 4-16

In [14]:
# 특성 벡터를 줄이는 방법 2
print(intermediate1.view(batch_size, -1).size())

# 특성 벡터를 줄이는 방법 2
print(torch.mean(intermediate1, dim=2).size())
# print(torch.max(intermediate1, dim=2).size())
# print(torch.sum(intermediate1, dim=2).size())

torch.Size([2, 80])
torch.Size([2, 16])


### 코드 4-22

전체 모델을 다시 만들지 않고 배치 정규화를 사용하는 방법만 소개합니다.

In [15]:
conv1 = nn.Conv1d(in_channels=one_hot_size, out_channels=16, kernel_size=3)
conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3)
conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)

conv1_bn = nn.BatchNorm1d(num_features=16)
conv2_bn = nn.BatchNorm1d(num_features=32)
    
intermediate1 = conv1_bn(F.relu(conv1(data)))
intermediate2 = conv2_bn(F.relu(conv2(intermediate1)))
intermediate3 = conv3(intermediate2)

print(intermediate1.size())
print(intermediate2.size())
print(intermediate3.size())

torch.Size([2, 16, 5])
torch.Size([2, 32, 3])
torch.Size([2, 64, 1])


노트: 배치 정규화는 배치와 시퀀스 차원에 대해 통곗값을 계산합니다. 다른 말로하면 `BatchNorm1d`에 입력되는 텐서의 크기는 `(B, C, L)`입니다(여기에서 B는 배치, C는 채널, L은 길이). 각 `(B, L)` 슬라이스마다 원점에 평균을 맞춥니다. 이는 공변량 변화(covariate shift)를 줄입니다.

In [16]:
intermediate2.mean(dim=(0, 2))

tensor([-2.9802e-08,  1.2418e-09,  0.0000e+00, -1.9868e-08, -9.9341e-09,
         9.9341e-09,  2.4835e-09, -9.9341e-09, -1.2418e-09, -1.9868e-08,
         1.8626e-09,  0.0000e+00, -1.2418e-09,  0.0000e+00,  1.9868e-08,
         9.3132e-10, -4.9671e-09, -9.9341e-09,  2.9802e-08,  9.9341e-09,
        -9.7013e-11,  9.9341e-09,  1.3970e-09,  0.0000e+00,  9.9341e-09,
         9.9341e-09,  1.9868e-08, -4.9671e-09, -1.2418e-09,  4.4703e-08,
         4.9671e-09, -5.9605e-08], grad_fn=<MeanBackward0>)

## 보너스 코드

4장에서 합성곱을 다룹니다. 아래 코드는 여러 하이퍼파라미터 설정으로 합성곱을 만듭니다.

In [17]:
x = torch.randn(1, 2, 3, 3)
describe(x)

conv1 = nn.Conv2d(in_channels=2, out_channels=1, kernel_size=2)
describe(conv1.weight)
describe(conv1(x))

Type: torch.FloatTensor
Shape/size: torch.Size([1, 2, 3, 3])
Values: 
tensor([[[[-1.3831,  0.5164,  0.2551],
          [-0.4873,  1.1319,  1.4091],
          [-1.0097,  1.3822,  2.5432]],

         [[-0.5137, -0.1112, -0.7682],
          [ 1.0231, -1.3065,  0.2210],
          [-0.3294,  0.6213,  2.1973]]]])
Type: torch.FloatTensor
Shape/size: torch.Size([1, 2, 2, 2])
Values: 
Parameter containing:
tensor([[[[-0.0395, -0.2742],
          [-0.1382,  0.0440]],

         [[-0.2400,  0.1503],
          [ 0.1100, -0.1167]]]], requires_grad=True)
Type: torch.FloatTensor
Shape/size: torch.Size([1, 1, 2, 2])
Values: 
tensor([[[[ 0.5734, -0.2716],
          [-0.4697, -0.1801]]]], grad_fn=<MkldnnConvolutionBackward>)


In [18]:
x = torch.randn(1, 1, 3, 3)
describe(x)

conv1 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=2)
describe(conv1.weight)
describe(conv1(x))

Type: torch.FloatTensor
Shape/size: torch.Size([1, 1, 3, 3])
Values: 
tensor([[[[-0.2682,  0.4390,  1.3682],
          [ 0.3038,  0.8558, -0.5000],
          [ 1.5619, -0.5929,  0.6817]]]])
Type: torch.FloatTensor
Shape/size: torch.Size([2, 1, 2, 2])
Values: 
Parameter containing:
tensor([[[[-0.0029, -0.3377],
          [-0.3707,  0.3836]]],


        [[[ 0.2779, -0.3865],
          [-0.1691,  0.4410]]]], requires_grad=True)
Type: torch.FloatTensor
Shape/size: torch.Size([1, 2, 2, 2])
Values: 
tensor([[[[ 0.4943, -0.5463],
          [-0.6703,  1.0738]],

         [[-0.1458, -0.9997],
          [-0.9996,  0.6042]]]], grad_fn=<MkldnnConvolutionBackward>)
